## Ilustratie van de werking van het RSA Algoritme

Belangrijk! DEZE CODE DIENT ENKEL TER ILLUSTRATIE!
NIET GEBRUIKEN VOOR DOELEINDEN WAARBIJ ECHTE VEILIGHEID VEREIST IS!

In [ ]:
def extended_euclid(a, b):
    """ a, b : 2 natuurlijk getallen.
    
        Berekent x, y en z zodat: ax + by = z en z = ggd(a,b)
    
        Returns: (x, y, z) zodat ax + by = z = ggd(a, b)
    """
    r0 = a ; r1 = b
    s0 = 1 ; s1 = 0
    t0 = 0 ; t1 = 1
    while r1 != 0:
        q = r0 // r1
        (r0, r1) = (r1, r0 - q*r1)
        (s0, s1) = (s1, s0 - q*s1)
        (t0, t1) = (t1, t0 - q*t1)
    return (s0, t0, r0)

In [ ]:
def inverse_mod(a, n):
    """ a, n : natuurlijke getallen.
    
        Berekent de inverse van a modulu n, i.e. bepaalt b zodanig dat
        a * b = 1 mod n.
        
        Returns: de inverse van a modulo n
    """
    (x, y, ggd) = extended_euclid(a, n)
    if ggd == 1:
        return x % n
    else:
        print("Error: inverse kan niet berekend worden!")

In [ ]:
def pow_mod(x, p, m):
    """ x, p, m: natuurlijke getallen
    
        Berekent x^p mod m op een efficiënte manier. Gebruikt hiervoor de halveringsmethode.
        
        Returns: x^p mod m
    """
    if p == 0:
        return 1
    elif p == 1:
        return x % m
    else:
        half = p // 2
        pow_half = pow_mod(x, half, m) # Recursieve oproep
        pow_half = (pow_half * pow_half) % m
        if p % 2 == 0:
            return pow_half
        else:
            return (pow_half * x) % m

In [ ]:
import random
def probably_prime_fermat(p, num_tries = 50):
    """ p: het te testen getal
        num_tries: het aantal pogingen, hoe meer pogingen hoe lager de kans 
        dat de test het foutieve antwoord geeft.
        
        Voor een priemgetal p getal dat a^(p-1) = 1 mod p voor alle getallen tussen 
        kleiner dan p.  Als men maw een getal a vindt waarvoor a^(p-1) niet gelijk is 
        aan 1 mod p, dan is het getal p zeker NIET priem.
        
        returns: True als geen bewijs werd gevonden dat p geen priemgetal is, False 
        anders.
    """
    for i in range(num_tries):
        a = random.randint(2, p-1)
        if pow_mod(a, p - 1, p) != 1:
            return False # zeker geen priemgetal
    # Waarschijnlijk priem maar kan verkeerd zijn
    return True    

## Genereer een groot priemgetal
## Start bij een groot (oneven) random getal. Probeer de reeks start, start + 2, start + 4, ...
## tot men een priemgetal vindt. Dit zou normaalgezien tamelijk snel moeten gebeuren.
def generate_prime(lengte, debug = False):
    """ lengte: de lengte (in grondtal 10) van het gewenste priemgetal.
    
        We starten bij een groot (oneven) random getal 'start' en proberen de
        reeks tart, start + 2, start + 4, ... tot we een getal vinden dat 
        volgens de test van Fermat een priemgetal is.
    
        returns: een getal dat waarschijnlijk priem is.
    """
    start = random.randint(10**lengte, 10**(lengte+1))
    if start % 2 == 0:
        start += 1
    if debug:
        num = 1 ## aantal getallen geprobeerd
    while not probably_prime_fermat(start):
        if debug:
            num += 1
        start += 2
    if debug:
        print("Aantal getallen getest:", num)
    return start

In [ ]:
def message_to_number(message):
    """ message: een string, de boodschap
    
        Zet een boodschap om naar een getal.
        Boodschap mag enkel bestaan uit letters en spaties
        De boodschap wordt omgezet naar hoofdletters alvorens de omzetting
        naar een getal gebeurt.
        
        returns: een getal dat de boodschap voorstelt
    """
    letters = " ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    number = ""
    for letter in message.upper():
        i = letters.index(letter)
        if (i < 10):
            number += "0" + str(i)
        else:
            number += str(i)
    return int(number)
## Omgekeerde bewerking van de vorige.
def number_to_message(number):
    """ number: een natuurlijk getal dat een boodschap voorstelt
        
        Zet een getal terug om naar de boodschap volgens hetzelfde coderingsschema 
        als hierboven.
        
        returns: een string die de boodschap voorstelt
    """
    letters = " ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    message = ""
    while number != 0:
        i = number % 100 
        if i < len(letters):
            message = letters[i] + message
        else:
            message += "?" # Om aan te duiden dat er iets fout ging
        number //= 100
    return message
              
number_to_message(message_to_number("A PRIME"))    

In [ ]:
boodschap = "A PRIME"
print(f"De boodschap {boodschap} wordt {message_to_number(boodschap)}")
m = message_to_number(boodschap)
print(f"Het getal {m} komt overeen met {number_to_message(m)}")

In [ ]:
# Grootste ontdekte priemgetal in 1951
priem1951 = 20988936657440586486151264256610222593863921
probably_prime_fermat(priem1951)

In [ ]:
def generate_rsa_key(lengte = 100):
    """ Determines a RSA keypair:
        
        Returns (n, e, d)                
    """
    p = generate_prime(lengte)
    q = generate_prime(lengte)
    n = p * q
    phi = (p-1)*(q-1)
    e = 65537 # voor de eenvoud kiezen we altijd dezelfde e
    (_,_, ggd) = extended_euclid(e, phi)
    if ggd != 1:
        return f"Error, ggd van e={e} en phi={phi} verschillend van 1"
    d = inverse_mod(e, phi)
    return (n, e, d)

In [ ]:
def encrypt_rsa(message, n, e):
    if isinstance(message, str):
        m = message_to_number(message)
    else:
        m = message # we gaan ervan uit dat het nu een getal is
    print(f"m = {m}")
    if m < n:
        c = pow_mod(m, e, n)
        return c
    else:
        print("Error: boodschap te groot")
def decrypt_rsa(c, d, n):
    m = pow_mod(c, d, n)
    message = number_to_message(m)
    return message

In [ ]:
(n, e, d) = generate_rsa_key(lengte = 2)
print(n, e, d)

In [ ]:
c = encrypt_rsa("HI", n, e)
print(f"Vercijferde boodschap {c}")
m = decrypt_rsa(c, d, n)
print(f"Ontcijferde boodschap {m}")
print("Ontcijferen met de verkeerde d")
for dd in [random.randint(2, n) for _ in range(10)]:
    print(f"Ontcijferde boodschap {decrypt_rsa(c,dd, n)}")

In [ ]:
def onderteken_boodschap(message, d, n):
    return encrypt_rsa(message, d, n)

In [ ]:
# Gebruikt door de verzender

## OPLETTEN: werkt niet opmdat de boodschap m1 te groot kan zijn (groter dan ontvanger_n)
#def onderteken_en_vercijfer_boodschap(message, verzender_d, verzender_n, ontvanger_e, ontvanger_n):   
#    m1 = onderteken_boodschap(message, verzender_d, verzender_n) ## Dit werkt niet.
#    c = encrypt_rsa(m1, ontvanger_e, ontvanger_n)
#    return c
# Gebruikt door de ontvanger
#def ontcijfer_gehandtekende_boodschap(c, ontvanger_d, ontvanger_n, verzender_e, verzender_n):
#    m = decrypt_rsa(c, ontvanger_d, ontvanger_n)
#    m1 = decrypt_rsa(m, verzender_e, verzender_n)
#    return number_to_message(m1)
    